# Harvesting Global River Data 

The purpose of this script is to download global data on river locations and names from Open Street Map

In [71]:
import osm2geojson 
import requests
import json
import fiona
from fiona.crs import from_epsg
import time
import geopandas as gdp
import pandas as pd

In [60]:
missing_bbox_list= [(50,0,55,10),(55,0,60,10),(40,0,45,10),(45,0,50,10),(40,10,45,20),(45,10,50,20)]
time_outs=[180,180,180,360,360,360,540,540,540]
rivers=[]

In [61]:
overpass_url = "https://overpass.openstreetmap.fr/api/interpreter"

In [62]:
template = """
[out:json]
[timeout:{time_out}];
way{bbox}[waterway=river]["name"];
(._;>;);out;
"""

In [63]:
for bbox in missing_bbox_list:    
    print("Getting rivers for: " + str(bbox))
    for attempt in range(10):                
        try: 
            overpass_query= template.format(time_out=time_outs[attempt],bbox=bbox)
            response = requests.get(overpass_url, params={'data': overpass_query,}, headers = {'User-agent': 'wriuser'})
            data = response.json()       
            geojson= osm2geojson.json2geojson(data)
            break
        except: 
            print("error " + str(response.status_code) + "- retrying in 3 minutes")
            time.sleep(180)                
    else:
        print("unable to fetch data for" + str(bbox))
        break            
    for way in geojson['features']: 
        rivers.append(way)
    print(str(len(geojson['features'])) + " rivers added. Total rivers: " + str(len(rivers)))

Getting rivers for: (50, 0, 55, 10)
https://overpass.openstreetmap.fr/api/interpreter?data=%0A%5Bout%3Ajson%5D%0A%5Btimeout%3A180%5D%3B%0Away%2850%2C+0%2C+55%2C+10%29%5Bwaterway%3Driver%5D%5B%22name%22%5D%3B%0A%28._%3B%3E%3B%29%3Bout%3B%0A
converting to geojson
14075 rivers added. Total rivers: 14075
Getting rivers for: (55, 0, 60, 10)
https://overpass.openstreetmap.fr/api/interpreter?data=%0A%5Bout%3Ajson%5D%0A%5Btimeout%3A180%5D%3B%0Away%2855%2C+0%2C+60%2C+10%29%5Bwaterway%3Driver%5D%5B%22name%22%5D%3B%0A%28._%3B%3E%3B%29%3Bout%3B%0A
converting to geojson
1572 rivers added. Total rivers: 15647
Getting rivers for: (40, 0, 45, 10)
https://overpass.openstreetmap.fr/api/interpreter?data=%0A%5Bout%3Ajson%5D%0A%5Btimeout%3A180%5D%3B%0Away%2840%2C+0%2C+45%2C+10%29%5Bwaterway%3Driver%5D%5B%22name%22%5D%3B%0A%28._%3B%3E%3B%29%3Bout%3B%0A
converting to geojson
8449 rivers added. Total rivers: 24096
Getting rivers for: (45, 0, 50, 10)
https://overpass.openstreetmap.fr/api/interpreter?data=%0A%5

In [64]:
schema = {'geometry': 'LineString', 'properties': {'Name':'str:80'}}
shapeout = "osm_rivers_to_add.shp"
with fiona.open(shapeout, 'w',crs=from_epsg(4326),driver='ESRI Shapefile', schema=schema) as output:
    for way in rivers:
        # the shapefile geometry use (lon,lat) 
        line = {'type': 'LineString', 'coordinates': way['geometry']['coordinates']}
        prop = {'Name': way['properties']['tags']['name']}
        try:
            output.write({'geometry': line, 'properties':prop})  
        except:
            print("could not add " + prop["Name"])

could not add Dijle
could not add Muhder Sieltief
could not add Muhder Sieltief
could not add Muhder Sieltief
could not add Rio Mare Foghe


In [68]:
to_add_gdf = gdp.read_file("osm_rivers_to_add.shp")
rivers_gdf = gdp.read_file("osm_rivers.shp")

In [73]:
rivers_gdf = pd.concat([to_add_gdf, rivers_gdf], ignore_index=True)

In [74]:
rivers_gdf.to_file(driver = 'ESRI Shapefile', filename = "osm_rivers_final.shp")

Spatial Join in ArcMap